# Lab 3: Agentic Search Tools


## 環境設定

In [1]:
from dotenv import load_dotenv
import json
import os
import re
import sys
import warnings
import boto3
from botocore.config import Config
warnings.filterwarnings("ignore")
import logging

# ローカルモジュールのインポート
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# 基本設定
logger = utils.set_logger()  # ロガーの設定
pp = utils.set_pretty_printer()  # 整形出力用のプリンターを設定

# .envファイルまたはSecret Managerから環境変数を読み込む
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")  # AWS地域を環境変数から取得
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)  # Tavily APIキーを取得

# Bedrockの設定
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={"max_attempts": 0})  # タイムアウトと再試行の設定

# Bedrockランタイムクライアントの作成
bedrock_rt = boto3.client("bedrock-runtime", region_name=aws_region, config=bedrock_config)

# 利用可能なモデルを確認するためのBedrockクライアントの作成
bedrock = boto3.client("bedrock", region_name=aws_region, config=bedrock_config)

[2025-07-09 18:54:25,669] p1141744 {utils.py:66} INFO - TAVILY_API_KEY variable correctly retrieved from the .env file.


In [3]:
from tavily import TavilyClient

# TavilyClientの初期化 - 環境変数から取得したAPIキーを使用
client = TavilyClient(api_key=tavily_ai_api_key)

# 検索の実行 - NVIDIAの新しいBlackwell GPUについての情報を検索
# include_answer=Trueを指定することで、検索結果だけでなく要約された回答も取得
result = client.search("NVIDIAの新しいBlackwell GPUには何が搭載されていますか？", include_answer=True)

# 回答を出力
# result["answer"]には、検索結果から生成された要約回答が含まれている
result["answer"]

'NVIDIAの新しいBlackwell GPUには、GDDR7メモリと第2世代Transformer Engineが搭載されています。最大30Gbpsの速度を持つGDDR7メモリが使用されています。'

## Regular search


In [4]:
# choose location (try to change to your own city!)

city = "Tokyo"

# 天気情報と旅行の推奨に関する質問を作成
# {city}は変数で、特定の都市名が代入される想定
# 最後の"weather.com"は情報源を指定している可能性がある
query = f"""現在の{city}の天気はどうですか？
今日そこへ旅行すべきでしょうか？
'weather.com'"""

> 注意: 例外が発生した場合に予想される結果を返すように検索が修正されました。大量の学生トラフィックにより、レート制限の例外が発生することがあります。

In [6]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

# DuckDuckGoの検索クライアントを初期化
ddg = DDGS()

def search(query, max_results=6):
    try:
        # DuckDuckGoで検索を実行し、指定された最大結果数を取得
        results = ddg.text(query, max_results=max_results)
        # 結果からURLのリストを返す
        return [i["href"] for i in results]
    except Exception as e:
        # 高トラフィック量によるDDGのレート制限などの例外が発生した場合
        print(f"DDGへのアクセスで例外が発生したため、事前に用意した結果を返します。")
        # 天気関連の検索結果をフォールバックとして提供
        results = [  # 高トラフィック量によるDDGのレート制限の場合に備えて
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results

# 検索を実行し、結果のURLを表示
for i in search(query):
    print(i)

https://www.youtube.com/
https://www.youtube.com/feed
https://support.google.com/youtube/?hl=en
https://music.youtube.com/
https://play.google.com/store/apps/details?id=com.google.android.youtube&hl=en-US
https://support.google.com/youtube/answer/3227660?hl=en&co=GENIE.Platform=Android


/tmp/ipykernel_1141744/552516275.py:7: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


In [7]:
def scrape_weather_info(url):
    """指定されたURLからコンテンツをスクレイピングする"""
    if not url:
        return "天気情報が見つかりませんでした。"
    
    # データを取得
    # User-Agentヘッダーを設定してブラウザからのリクエストに見せかける
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    
    # レスポンスのステータスコードをチェック
    if response.status_code != 200:
        return "ウェブページの取得に失敗しました。"
    
    # 結果を解析
    # BeautifulSoupを使用してHTMLを解析し、操作可能なオブジェクトに変換
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

> 注意: これは長い出力を生成します。簡単に確認した後、スクロールを避けるために右クリックしてセル出力をクリアすることをお勧めします。

In [9]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:500])  # limit long outputs

Website: https://www.credit-agricole.fr/particulier/conseils/magazine/tout-un-mag/vos-comptes-en-ligne-a-vous-la-liberte.html


<body>
<div class="SkipLinksBanner sr-only">
<nav aria-label="AccÃ¨s rapide" role="navigation">
<ul class="skiplinks">
<li>
<a class="npc-msl-link inverted npc-msl-small" href="#header" id="byPassAllerAuHeader">Aller au Menu</a>
</li>
<li>
<a class="npc-msl-link inverted npc-msl-small" href="#content" id="byPassAllerAuContenu">Aller au Contenu</a>
</li>
<li>
<a class="npc-msl-link inverted npc-msl-small" href="#footer" id="byPassAllerAuFooter">Aller au Pied de page</a>
</li>
</ul>
</nav>
</div>



In [10]:
# extract text
weather_data = []
for tag in soup.find_all(["h1", "h2", "h3", "p"]):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r"\s+", " ", weather_data)

print(f"Website: {url}\n\n")
print(weather_data)

Website: https://www.credit-agricole.fr/particulier/conseils/magazine/tout-un-mag/vos-comptes-en-ligne-a-vous-la-liberte.html


Malheureusement, votre configuration de navigation actuelle ne vous permet pas de naviguer dans de bonnes conditions. Vous ne pourrez pas profiter de toutes les fonctionnalitÃ©s de notre site ni accÃ©der Ã votre espace client. FIN DE CONNEXION Vous n'Ãªtes actuellement plus connectÃ© Ã CrÃ©dit Agricole en Ligne. Si vous souhaitez poursuivre la consultation de vos comptes, nous vous invitons Ã vous identifier Ã nouveau. Vous n'Ãªtes actuellement plus connectÃ© Ã CrÃ©dit Agricole en Ligne Vous n'Ãªtes actuellement plus connectÃ© Ã CrÃ©dit Agricole en Ligne. Si vous souhaitez poursuivre la consultation de vos comptes, nous vous invitons Ã vous identifier Ã nouveau. Vous n'Ãªtes actuellement plus connectÃ© Ã CrÃ©dit Agricole en Ligne. Si vous souhaitez poursuivre la consultation de vos comptes, nous vous invitons Ã vous identifier Ã nouveau. Vous n'Ãªtes actuellem

## Agentic Search


In [11]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1752087784, 'localtime': '2025-07-10 04:03'}, 'current': {'last_updated_epoch': 1752087600, 'last_updated': '2025-07-10 04:00', 'temp_c': 27.2, 'temp_f': 81.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 8.9, 'wind_kph': 14.4, 'wind_degree': 190, 'wind_dir': 'S', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 84, 'cloud': 25, 'feelslike_c': 30.5, 'feelslike_f': 87.0, 'windchill_c': 27.0, 'windchill_f': 80.6, 'heatindex_c': 30.1, 'heatindex_f': 86.2, 'dewpoint_c': 23.0, 'dewpoint_f': 73.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 12.5, 'gust_kph': 20.1}}


In [12]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(
    formatted_json, lexers.JsonLexer(), formatters.TerminalFormatter()
)

print(colorful_json)

{
    "location": {
        "name": "Tokyo",
        "region": "Tokyo",
        "country": "Japan",
        "lat": 35.6895,
        "lon": 139.6917,
        "tz_id": "Asia/Tokyo",
        "localtime_epoch": 1752087784,
        "localtime": "2025-07-10 04:03"
    },
    "current": {
        "last_updated_epoch": 1752087600,
        "last_updated": "2025-07-10 04:00",
        "temp_c": 27.2,
        "temp_f": 81.0,
        "is_day": 0,
        "condition": {
            "text": "Clear",
            "icon": "//cdn.weatherapi.com/weather/64x64/night/113.png",
            "code": 1000
        },
        "wind_mph": 8.9,
        "wind_kph": 14.4,
        "wind_degree": 190,
        "wind_dir": "S",
        "pressure_mb": 1011.0,
        "pressure_in": 29.85,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 84,
        "cloud": 25,
        "feelslike_c": 30.5,
        "feelslike_f": 87.0,
        "windchill_c": 27.0,
        "windchill_f": 80.6,
        "heatindex_c": 3